<a href="https://colab.research.google.com/github/Faithkym7/Special-topics---kiswahili-to-english-translator/blob/main/Kiswahili__to_english_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [1]:
!pip install gdown
!gdown --id 1B51schTvzz3DYyQ03s-KECs-bSaEzhjR
!gdown --id 1eBiew8h6O3RAeqwXd4xwrVLp9w--umNZ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1B51schTvzz3DYyQ03s-KECs-bSaEzhjR
To: /content/english2.csv
100% 13.7k/13.7k [00:00<00:00, 30.9MB/s]
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1eBiew8h6O3RAeqwXd4xwrVLp9w--umNZ
To: /content/swahili2.csv
100% 11.7k/11.7k [00:00<00:00, 29.1MB/s]


In [6]:
import pandas as pd
import re

# Read the data from two separate files

kiswahili_data = pd.read_csv('swahili2.csv', encoding='ISO-8859-1', header=None, names=['Swahili sample sentences'])
english_data = pd.read_csv('english2.csv', encoding='ISO-8859-1', header=None, names=['English sample sentence(s)'])


# Remove any non-alphanumeric characters from the sentences
kiswahili_data['Swahili sample sentences'] = kiswahili_data['Swahili sample sentences'].astype(str)
kiswahili_data['Swahili sample sentences'] = kiswahili_data['Swahili sample sentences'].apply(lambda x: re.sub(r'\W+', ' ', x).strip().lower())
english_data['English sample sentence(s)'] = english_data['English sample sentence(s)'].astype(str)
english_data['English sample sentence(s)'] = english_data['English sample sentence(s)'].apply(lambda x: re.sub(r'\W+', ' ', x).strip().lower())




# Combine the two datasets into one
kiswahili_data = kiswahili_data.reset_index(drop=True)
english_data = english_data.reset_index(drop=True)
parallel_data = pd.concat([kiswahili_data, english_data], axis=1)



# Split the data into training and validation sets
train_data = parallel_data.sample(frac=0.8, random_state=42)
val_data = parallel_data.drop(train_data.index)

# Save the datasets to files
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)


In [7]:
df = pd.read_csv('train_data.csv')
print(df.head(5))
df = pd.read_csv('val_data.csv')
print(df.head(50))

                            Swahili sample sentences  \
0  lengo langu ni kuelimisha watu kuwezesha mwana...   
1                                namjua sijui unajua   
2                 nimepotea mumepotea la hatujapotea   
3  tuliamua kwenda mjomba wangu aliamua kuanza bi...   
4                          anapendeza leo umependeza   

                          English sample sentence(s)  
0  my goal is to educate people empowering a woma...  
1        i know this person i don t know do you know  
2            i m lost are you lost no we re not lost  
3  we decided to go my uncle decided to start a b...  
4               she looks great today you look great  
                            Swahili sample sentences  \
0  mwaka ijayo nitakuwa mwanafunzi nilikuwa na nj...   
1  naomba ndizi na nyama tunaomba samaki naomba m...   
2            tunaweza kuwasiliana kwa kiswahili sasa   
3                        lengo langu ni kuwasiliiana   
4  nitapata maji baadaye ninahitaji kupata chakul... 

In [35]:
!pip uninstall keras tensorflow
!pip install keras==2.4.3 tensorflow==2.4.1


Found existing installation: keras 2.11.0
Uninstalling keras-2.11.0:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/keras-2.11.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/keras/*
Proceed (Y/n)? Y
  Successfully uninstalled keras-2.11.0
Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.11.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow

In [41]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import numpy as np
import pandas as pd

# Load the training and validation data
train_data = pd.read_csv('train_data.csv')
val_data = pd.read_csv('val_data.csv')

# Define a function to get the length of a string or return 0 for NaN values
def get_length(x):
    try:
        return len(x)
    except TypeError:
        return 0

# Define the maximum length of the input and output sequences
max_encoder_seq_length = max(train_data['Swahili sample sentences'].apply(get_length))
max_decoder_seq_length = max(train_data['English sample sentence(s)'].apply(get_length))



# Define the number of encoder and decoder tokens
encoder_num_tokens = 10000
decoder_num_tokens = 10000

train_data.dropna(inplace=True)


# Initialize the tokenizers for the encoder and decoder
encoder_tokenizer = Tokenizer(num_words=encoder_num_tokens, filters='')
encoder_tokenizer.fit_on_texts(train_data['Swahili sample sentences'])
decoder_tokenizer = Tokenizer(num_words=decoder_num_tokens, filters='')
decoder_tokenizer.fit_on_texts(train_data['English sample sentence(s)'])

# Convert the text data to integer sequences
encoder_input_data = encoder_tokenizer.texts_to_sequences(train_data['Swahili sample sentences'])
decoder_input_data = decoder_tokenizer.texts_to_sequences(train_data['English sample sentence(s)'])
decoder_target_data = decoder_tokenizer.texts_to_sequences(train_data['English sample sentence(s)'].apply(lambda x: ' '.join(x.split()[1:] + ['<eos>'])))

# Pad the sequences to a fixed length
encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_encoder_seq_length, padding='post')
decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_decoder_seq_length, padding='post')
decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_decoder_seq_length, padding='post')

# Define the model architecture
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(encoder_num_tokens, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(decoder_num_tokens, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(decoder_num_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Print the model summary
model.summary()

# Train the model
batch_size = 32
epochs = 10
checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True)
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[checkpoint])


Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_23 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_15 (Embedding)       (None, None, 256)    2560000     ['input_22[0][0]']               
                                                                                                  
 embedding_16 (Embedding)       (None, None, 256)    2560000     ['input_23[0][0]']               
                                                                                           

In [55]:
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the trained model and tokenizer
model = load_model('model_weights.h5')
encoder_tokenizer = Tokenizer(num_words=10000, filters='')
encoder_tokenizer.fit_on_texts([' '.join(['<sos>', 'hello', '<eos>'])])  # Dummy input to initialize the tokenizer

# Define the maximum length of the input and output sequences
max_encoder_seq_length = 20
max_decoder_seq_length = 20

# Define a function to generate the English translation
def translate_kiswahili_to_english(input_text):
    # Convert the input text to a sequence of integer-encoded tokens
    input_seq = encoder_tokenizer.texts_to_sequences([input_text])[0]
    # Pad the input sequence to a fixed length
    input_seq = pad_sequences([input_seq], maxlen=max_encoder_seq_length, padding='post')
    # Initialize the decoder input sequence with the <sos> token
    decoder_input_seq = np.zeros((1, max_decoder_seq_length))
    decoder_input_seq[0, 0] = encoder_tokenizer.word_index['<sos>']
    # Loop through the decoder until we reach the maximum sequence length or generate the <eos> token
    for i in range(1, max_decoder_seq_length):
        # Predict the next token in the sequence
        output_tokens = model.predict([input_seq, decoder_input_seq]).argmax(axis=-1)
        # If we generate the <eos> token, stop decoding
        if output_tokens[0, i] == encoder_tokenizer.word_index['<eos>']:
            break
        # Add the predicted token to the decoder input sequence
        decoder_input_seq[0, i] = output_tokens[0, i]
    # Convert the output sequence of integer-encoded tokens to English text
    output_text = ' '.join(encoder_tokenizer.index_word[i] for i in decoder_input_seq[0] if i > 0)
    return output_text

# Test the model with a sample input
input_text = 'Habari yako'
output_text = translate_kiswahili_to_english(input_text)
print('Input text:', input_text)
print('Translated text:', output_text)


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7fe82375dca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7fe82375dca0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 32ms/step
Input text: Habari yako
Translated text: <sos>


In [57]:
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

# Load the trained model and tokenizer
model = load_model('model_weights.h5')
encoder_tokenizer = Tokenizer(num_words=10000, filters='')
encoder_tokenizer.fit_on_texts([' '.join(['<sos>', 'hello', '<eos>'])])  # Dummy input to initialize the tokenizer

# Define the maximum length of the input and output sequences
max_encoder_seq_length = 20
max_decoder_seq_length = 20

# Define a function to generate the English translation
@tf.autograph.experimental.do_not_convert
def predict_function(input_seq, decoder_input_seq):
    return model.predict([input_seq, decoder_input_seq])

def translate_kiswahili_to_english(input_text):
    # Convert the input text to a sequence of integer-encoded tokens
    input_seq = encoder_tokenizer.texts_to_sequences([input_text])[0]
    # Pad the input sequence to a fixed length
    input_seq = pad_sequences([input_seq], maxlen=max_encoder_seq_length, padding='post')
    # Initialize the decoder input sequence with the <sos> token
    decoder_input_seq = np.zeros((1, max_decoder_seq_length))
    decoder_input_seq[0, 0] = encoder_tokenizer.word_index['<sos>']
    # Loop through the decoder until we reach the maximum sequence length or generate the <eos> token
    for i in range(1, max_decoder_seq_length):
        # Predict the next token in the sequence
        output_tokens = predict_function(input_seq, decoder_input_seq).argmax(axis=-1)
        # If we generate the <eos> token, stop decoding
        if output_tokens[0, i] == encoder_tokenizer.word_index['<eos>']:
            break
        # Add the predicted token to the decoder input sequence
        decoder_input_seq[0, i] = output_tokens[0, i]
    # Convert the output sequence of integer-encoded tokens to English text
    output_text = ' '.join(encoder_tokenizer.index_word[i] for i in decoder_input_seq[0] if i > 0)
    return output_text

# Test the model with a sample input
input_text = 'Nitakufahamisha'
output_text = translate_kiswahili_to_english(input_text)
print('Input text:', input_text)
print('Translated text:', output_text)


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7fe823c30b80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function Model.make_predict_function.<locals>.predict_function at 0x7fe823c30b80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 30ms/step
Input text: Nitakufahamisha
Translated text: <sos>
